In [1]:
from dotenv import load_dotenv

load_dotenv()

False

In [2]:
from camel.logger import get_logger
from camel.datasets import StaticDataset, FewShotGenerator


logger = get_logger(__name__)



data = [
    {
        "question": "Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).",
        "final_answer": "0",
        "rationale": '''from sympy import symbols, Eq, solve
a, b = symbols('a b')
eq1 = Eq(2*a + 3, -3)
eq2 = Eq(-7, -4 - b)
sol_a = solve(eq1, a)[0]
sol_b = solve(eq2, b)[0]
result = sol_a + sol_b
print(result)'''
    },
    {
        "question": "What is the degree of the polynomial $(4 +5x^3 +100 +2\\pi x^4 + \\sqrt{10}x^4 +9)$?",
        "final_answer": "4",
        "rationale": '''from sympy import symbols, pi, sqrt, degree
x = symbols('x')
poly = 4 + 5*x**3 + 100 + 2*pi*x**4 + sqrt(10)*x**4 + 9
deg = degree(poly, x)
print(deg)'''
    },
    {
        "question": "Evaluate $\\left\\lceil3\\left(6-\\frac12\\right)\\right\\rceil$.",
        "final_answer": "17",
        "rationale": '''from sympy import Rational, ceiling
expr = 3 * (6 - Rational(1,2))
result = ceiling(expr)
print(result)'''
    },
    {
        "question": "Sam is hired for a 20-day period. On days that he works, he earns $\\$$60. For each day that he does not work, $\\$$30 is subtracted from his earnings. At the end of the 20-day period, he received $\\$$660. How many days did he not work?",
        "final_answer": "6",
        "rationale": '''from sympy import symbols, Eq, solve
x, y = symbols('x y')
eq1 = Eq(x + y, 20)
eq2 = Eq(60*x - 30*y, 660)
solution = solve((eq1, eq2), (x, y))
print(solution[y])'''
    },
    {
        "question": "Find the center of the circle with equation $x^2 - 6x + y^2 + 2y = 9$.",
        "final_answer": "(3, -1)",
        "rationale": '''from sympy import symbols, Eq, solve
x, y = symbols('x y')
# Complete the square for x: x^2 - 6x = (x - 3)^2 - 9
# Complete the square for y: y^2 + 2y = (y + 1)^2 - 1
# Equation becomes: (x - 3)^2 - 9 + (y + 1)^2 - 1 = 9
# Simplify: (x - 3)^2 + (y + 1)^2 = 19
h = 3  # x-coordinate of center
k = -1  # y-coordinate of center
result = (h, k)
print(result)'''
    },
    {
        "question": "What are all values of $p$ such that for every $q>0$, we have $$\\frac{3(pq^2+p^2q+3q^2+3pq)}{p+q}>2p^2q?$$ Express your answer in interval notation in decimal form.",
        "final_answer": "[0, 3)",
        "rationale": '''from sympy import symbols, solve, GreaterThan
p = symbols('p')
# After simplification: 3(p + 3) > 2*p**2 for all q > 0, assuming p >= 0
expr = 2*p**2 - 3*p - 9  # Solve 2p^2 - 3p - 9 < 0
solutions = solve(expr < 0, p)
# Roots are -1.5 and 3, negative between them, intersect with p >= 0
result = "[0, 3)"  # Hardcoded interval based on solution and p >= 0
print(result)'''
    },
    {
        "question": "If $x = 2$ and $y = 5$, then what is the value of $\\frac{x^4+2y^2}{6}$ ?",
        "final_answer": "11",
        "rationale": '''from sympy import symbols
x, y = symbols('x y')
expr = (x**4 + 2*y**2) / 6
result = expr.subs({x: 2, y: 5})
print(result)'''
    },
    {
        "question": "Tim wants to invest some money in a bank which compounds quarterly with an annual interest rate of $7\\%$. To the nearest dollar, how much money should he invest if he wants a total of $\\$60,\\!000$ at the end of $5$ years?",
        "final_answer": "42409",
        "rationale": '''from sympy import symbols, floor
P = symbols('P')
A = 60000
r = 0.07
n = 4
t = 5
eq = Eq(A, P * (1 + r/n)**(n*t))
P_value = solve(eq, P)[0]
result = floor(P_value.evalf())  # Round down to nearest dollar
print(result)'''
    },
    {
        "question": "Berengere and her American foreign-exchange student Emily are at a bakery in Paris that accepts both euros and American dollars. They want to buy a cake, but neither of them has enough money. If the cake costs 6 euros and Emily has an American five-dollar bill, how many euros does Berengere need to contribute to the cost of the cake if 1 euro = 1.25 USD?",
        "final_answer": "2",
        "rationale": '''from sympy import symbols, Eq, solve
B = symbols('B')
# 1 euro = 1.25 USD, so 5 USD = 5 / 1.25 euros
emily_contribution = 5 / 1.25  # 4 euros
total_cost = 6
eq = Eq(emily_contribution + B, total_cost)
result = solve(eq, B)[0]
print(int(result))'''
    },
    {
        "question": "Simplify $\\sqrt[3]{1+8} \\cdot \\sqrt[3]{1+\\sqrt[3]{8}}$.",
        "final_answer": "3",
        "rationale": '''from sympy import cbrt
# sqrt[3]{1 + 8} = cbrt(9)
# sqrt[3]{8} = 2, so sqrt[3]{1 + sqrt[3]{8}} = cbrt(1 + 2) = cbrt(3)
result = cbrt(9) * cbrt(3)  # cbrt(9 * 3) = cbrt(27) = 3
print(result)'''
    }
]

seed_dataset = StaticDataset(data)

In [3]:
from camel.verifiers import PythonVerifier

verifier = PythonVerifier(required_packages=["sympy"])
await verifier.setup(uv=True)

In [4]:
import os
from getpass import getpass
# Prompt for the OpenAI API key securely
openai_api_key = getpass('Enter your OpenAI API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

In [5]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.configs import ChatGPTConfig


model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig().as_dict(),
)
generator = FewShotGenerator(
    seed_dataset=seed_dataset, verifier=verifier, model=model
)

In [ ]:
await generator.generate_new(n=20)

2025-03-28 12:19:45,117 - camel.camel.datasets.few_shot_generator - WARNING - Verifier issue: Verifier unsuccessful, response: status=<VerificationOutcome.ERROR: 'error'> result='' duration=0.1460418701171875 timestamp=datetime.datetime(2025, 3, 28, 12, 19, 45, 116894) metadata={'attempt': 1} error_message='Solution code error:\nTraceback (most recent call last):\n  File "/tmp/tmpktc3qvu0.py", line 1, in <module>\n    from sympy import symbols, Vertex, Rational\nImportError: cannot import name \'Vertex\' from \'sympy\' (/tmp/tmpwpmx_15m/lib/python3.11/site-packages/sympy/__init__.py)', retrying... (1/10)
2025-03-28 12:19:52,246 - camel.camel.datasets.base_generator - ERROR - Error writing to file /home/stranger/CODE/camel/docs/cookbooks/loong: [Errno 21] Is a directory: '/home/stranger/CODE/camel/docs/cookbooks/loong'


IsADirectoryError: [Errno 21] Is a directory: '/home/stranger/CODE/camel/docs/cookbooks/loong'

In [ ]:
generator.save_to_jsonl(file_path="data.jsonl")

In [10]:
from pathlib import Path

proposal_synthetic_data = StaticDataset(Path("data.jsonl"))

In [13]:
from camel.agents import ChatAgent

agent = ChatAgent(model=model, system_message= "Use chain of thought reasoning to come up with a solution to the following question: ")

In [ ]:
agent_outputs = []

for i in range(len(proposal_synthetic_data)):
    dp = proposal_synthetic_data[i]
    question = dp.question

    agent_output = agent.step(question).msgs[0]
    print(agent_output)




BaseMessage(role_name='Assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='To find the maximum value of the function \\( f(x) = -2x^2 + 8x + 3 \\), we can start by determining the vertex of the quadratic function, as a parabola opens downwards (since the coefficient of \\( x^2 \\), which is -2, is negative) and thus will have a maximum point at its vertex.\n\nThe x-coordinate of the vertex for a quadratic function \\( ax^2 + bx + c \\) can be given by the formula:\n\n\\[\nx = -\\frac{b}{2a}\n\\]\n\nFor our function, we have:\n- \\( a = -2 \\)\n- \\( b = 8 \\)\n\nNow substituting these values into the formula:\n\n\\[\nx = -\\frac{8}{2(-2)}\n\\]\n\\[\nx = -\\frac{8}{-4} = 2\n\\]\n\nNext, we find the maximum value by substituting \\( x = 2 \\) back into the function \\( f(x) \\):\n\n\\[\nf(2) = -2(2)^2 + 8(2) + 3\n\\]\n\nCalculating \\( (2)^2 \\):\n\n\\[\n(2)^2 = 4\n\\]\n\nNow substituting this into \\( f(2) \\):\n\n\\[\nf(2) = -2(4) + 8(2) + 3\n\\]\n\\[\nf(2) 